In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from scipy import stats
from scipy.stats import shapiro, levene, f_oneway, kruskal, mannwhitneyu
from scipy.stats import f_oneway
from itertools import combinations
import seaborn as sns
from factor_analyzer import FactorAnalyzer
import glob
import pingouin as pg
from scipy.stats import ttest_ind 


In [3]:
def testing_for_assumptions_by_group(df, score_col="IEQ Score", group_col="Group"):
    print(f"# Testing assumptions for {score_col} #\n")

    groups = df[group_col].unique()
    group_data = {g: df[df[group_col] == g][score_col].dropna() for g in groups}

    # 1. Shapiro–Wilk Test for Normality
    print("## Shapiro–Wilk Test for Normality ##")
    normality = {}
    for group, data in group_data.items():
        print(f"Group {group}:")
        if len(data) < 3:
            print(f"  ⚠️ Not enough data (n={len(data)}) to test normality.\n")
            normality[group] = False
            continue
        stat, p = shapiro(data)
        normal = p > 0.05
        normality[group] = normal
        if normal:
            print(f"  ✔️ Stat = {stat:.4f}, p = {p:.4f} → Data appears normal\n")
        else:
            print(f"  ❌ Stat = {stat:.4f}, p = {p:.4f} → Data not normal\n")

    print("## Levene’s Test for Equal Variances ##")
    data_lists = [d for d in group_data.values() if len(d) >= 2]
    if len(data_lists) >= 2:
        stat, p = levene(*data_lists)
        equal_var = p > 0.05
        if equal_var:
            print(f"  ✔️ Stat = {stat:.4f}, p = {p:.4f} → Variances are equal\n")
        else:
            print(f"  ❌ Stat = {stat:.4f}, p = {p:.4f} → Variances are not equal\n")
    else:
        equal_var = False
        print("  ⚠️ Not enough groups with sufficient data to test variance\n")

    print(f"# Final Group Comparison Test for {score_col} #")
    normal_all = all(normality.values())
    sufficient_groups = len([group_data[g] for g in groups if len(group_data[g]) >= 2]) > 2
    if normal_all and equal_var:
        print("✅ All assumptions met. Running ANOVA:")
        if(sufficient_groups):
            stat, p = f_oneway(*[group_data[g] for g in groups if len(group_data[g]) >= 2])
            print(f"  ✔️ ANOVA Stat = {stat:.4f}, p = {p:.4f}")
        return 1
    else:
        print("❌ Assumptions not met. Using non-parametric Kruskal–Wallis:")
        if(sufficient_groups):
            stat, p = kruskal(*[group_data[g] for g in groups if len(group_data[g]) >= 2])
            print(f"  ✔️ Kruskal–Wallis Stat = {stat:.4f}, p = {p:.4f}")
        return 0


In [4]:
def analyze_groups_anova(df, score_col, group_col):
    # Extract data for each group
    groups = df[group_col].unique()
    group_data = [df[df[group_col] == group][score_col].dropna() for group in groups]
    
    # Combine all groups into one dataset
    all_data = pd.concat(group_data)
    grand_mean = all_data.mean()

    # Calculate Sum of Squares Between (SS_between)
    SS_between = sum(len(group) * (group.mean() - grand_mean)**2 for group in group_data)

    # Calculate Sum of Squares Within (SS_within)
    SS_within = sum(sum((group - group.mean())**2) for group in group_data)

    # Calculate Total Sum of Squares (SS_total)
    SS_total = SS_between + SS_within

    # Calculate Eta Squared (η²)
    eta_squared = SS_between / SS_total

    # Perform one-way ANOVA
    stat, p = f_oneway(*group_data)

    # Print results
    print("SS stands for Sum of Squares")
    print(f"SS_between: {SS_between:.4f}")
    print(f"SS_within: {SS_within:.4f}")
    print(f"SS_total: {SS_total:.4f}")
    print(f"Effect Size (Eta Squared, η²): {eta_squared:.4f}")
    print(f"ANOVA Results: F-statistic = {stat:.4f}, p-value = {p:.4f}")
    if p < 0.05:
        print("There is a significant difference between the groups.")
    else:
        print("No significant difference between the groups.")

    # Plot the scores for each group
    plt.figure(figsize=(8, 6))
    boxplot = plt.boxplot(group_data, labels=groups, patch_artist=True, showmeans=True)
    plt.title(f"{score_col} by {group_col}")

    # Assign different colors to each group
    colors = ['lightblue', 'lightgreen', 'lightcoral']
    for box, color in zip(boxplot['boxes'], colors):
        box.set_facecolor(color)

    plt.ylabel(score_col)
    plt.xlabel(group_col)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

    # Return results as a dictionary
    return {
        "SS_between": SS_between,
        "SS_within": SS_within,
        "SS_total": SS_total,
        "eta_squared": eta_squared,
        "anova_stat": stat,
        "anova_p_value": p
    }

In [5]:
def analyze_kruskal_test(df, score_col, group_col):
    groups = df[group_col].unique()
    group_data = [df[df[group_col] == group][score_col].dropna() for group in groups]
    stat, p = kruskal(*group_data)
    print(f"Kruskal-Wallis test statistic = {stat:.4f}, p-value = {p:.4f}")
    if p < 0.05:
        print("There is a significant difference between the groups.")
    else:
        print("No significant difference between the groups.")
    plt.figure(figsize=(8, 6))
    boxplot = plt.boxplot(group_data, labels=groups, patch_artist=True, showmeans=True)
    plt.title(f"{score_col} by {group_col} (Kruskal-Wallis Test)")
    colors = ['lightblue', 'lightgreen', 'lightcoral']
    for box, color in zip(boxplot['boxes'], colors):
        box.set_facecolor(color)

    plt.ylabel(score_col)
    plt.xlabel(group_col)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

    # Return results as a dictionary
    return {
        "kruskal_stat": stat,
        "kruskal_p_value": p
    }

In [6]:
# Define the folder containing the CSV files
data_folder = "Data"

# Initialize an empty list to store individual DataFrames
dataframes = []

# Iterate through all CSV files in the folder
for file_path in glob.glob(os.path.join(data_folder, "*.csv")):
    # Extract the file name without the folder path
    file_name = os.path.basename(file_path)
    
    # Extract participant type (text before .csv) and participant number (the only number in the file name)
    participant_type = file_name.split('_')[-1].split('.')[0]
    participant_number = ''.join(filter(str.isdigit, file_name))
    
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path, sep=';')
    
    # Add columns for participant type and participant number
    df['Group'] = participant_type
    df['Participant_id'] = participant_number
    
    # Append the DataFrame to the list
    dataframes.append(df)

print(f"Number of CSV files processed: {len(dataframes)}")
# Concatenate all DataFrames into one
metrics_pd = pd.concat(dataframes, ignore_index=True)
metrics_pd


# Display the updated DataFrame
metrics_pd

Number of CSV files processed: 8


,Score,Time,Distance,Group,Participant_id
0,-11.961150,9.986407,1.325559,HandTracking,1
1,-22.236070,15.201210,1.222435,HandTracking,1
2,-33.930440,20.864090,1.468172,HandTracking,1
3,-15.773620,11.837020,1.399721,HandTracking,1
4,6.770551,0.740310,1.165887,HandTracking,1
...,...,...,...,...,...
75,3.004337,3.101566,0.528355,Mouse,4
76,2.769711,3.221231,0.525217,Mouse,4
77,2.502146,3.299644,0.599045,Mouse,4
78,0.717627,4.198405,0.590375,Mouse,4


In [7]:
from scipy.stats import ttest_ind, mannwhitneyu

def analyze_two_groups(df, score_col, group_col):
    groups = df[group_col].unique()
    if len(groups) != 2:
        raise ValueError("This function is only for comparing two groups.")

    # Extract data for the two groups
    group1_data = df[df[group_col] == groups[0]][score_col].dropna()
    group2_data = df[df[group_col] == groups[1]][score_col].dropna()

    # Test for normality and equal variances
    _, p_normal1 = shapiro(group1_data)
    _, p_normal2 = shapiro(group2_data)
    _, p_levene = levene(group1_data, group2_data)

    if p_normal1 > 0.05 and p_normal2 > 0.05 and p_levene > 0.05:
        # Use t-test if assumptions are met
        stat, p = ttest_ind(group1_data, group2_data)
        test_used = "t-test"
    else:
        # Use Mann-Whitney U test if assumptions are not met
        stat, p = mannwhitneyu(group1_data, group2_data, alternative='two-sided')
        test_used = "Mann-Whitney U"

    print(f"{test_used} Results: Statistic = {stat:.4f}, p-value = {p:.4f}")
    return {
        "test_used": test_used,
        "statistic": stat,
        "p_value": p
    }

In [8]:
print(metrics_pd.columns)
results = []
for col in metrics_pd.columns:
    print(f"\n\n\n### Testing for {col} ### \n")
    if col == "Group" or col == "Participant_id":
        continue
    result = testing_for_assumptions_by_group(metrics_pd, score_col=col, group_col="Group")
    results.append({"Metric": col, "Assumptions_Met": bool(result)})

# Convert the results into a DataFrame for summary
summary_df = pd.DataFrame(results)
summary_df

Index(['Score', 'Time', 'Distance', 'Group', 'Participant_id'], dtype='object')



### Testing for Score ### 

# Testing assumptions for Score #

## Shapiro–Wilk Test for Normality ##
Group HandTracking:
  ❌ Stat = 0.7273, p = 0.0000 → Data not normal

Group Mouse:
  ✔️ Stat = 0.9628, p = 0.2086 → Data appears normal

## Levene’s Test for Equal Variances ##
  ❌ Stat = 16.6246, p = 0.0001 → Variances are not equal

# Final Group Comparison Test for Score #
❌ Assumptions not met. Using non-parametric Kruskal–Wallis:



### Testing for Time ### 

# Testing assumptions for Time #

## Shapiro–Wilk Test for Normality ##
Group HandTracking:
  ❌ Stat = 0.7261, p = 0.0000 → Data not normal

Group Mouse:
  ✔️ Stat = 0.9703, p = 0.3671 → Data appears normal

## Levene’s Test for Equal Variances ##
  ❌ Stat = 16.2210, p = 0.0001 → Variances are not equal

# Final Group Comparison Test for Time #
❌ Assumptions not met. Using non-parametric Kruskal–Wallis:



### Testing for Distance ### 

# Testing

,Metric,Assumptions_Met
0,Score,False
1,Time,False
2,Distance,False


In [9]:
score_test = analyze_two_groups(metrics_pd, score_col="Score", group_col="Group")
time_test = analyze_two_groups(metrics_pd, score_col="Time", group_col="Group")
distance_test = analyze_two_groups(metrics_pd, score_col="Distance", group_col="Group")

group_stats = metrics_pd.groupby("Group").agg({
    "Score": ["mean", "std", "max", "min"],
    "Time": ["mean", "std", "max", "min"],
    "Distance": ["mean", "std", "max", "min"]
}).reset_index()

# Flatten the MultiIndex columns
group_stats.columns = ['Group', 'Score_mean', 'Score_std', 'Score_max', 'Score_min',
                       'Time_mean', 'Time_std', 'Time_max', 'Time_min',
                       'Distance_mean', 'Distance_std', 'Distance_max', 'Distance_min']
                    # Create separate DataFrames for Score, Time, and Distance
score_df = metrics_pd[["Group", "Score"]].groupby("Group").agg(["mean", "std", "max", "min"]).reset_index()
score_df.columns = ["Group", "Score_mean", "Score_std", "Score_max", "Score_min"]

time_df = metrics_pd[["Group", "Time"]].groupby("Group").agg(["mean", "std", "max", "min"]).reset_index()
time_df.columns = ["Group", "Time_mean", "Time_std", "Time_max", "Time_min"]

distance_df = metrics_pd[["Group", "Distance"]].groupby("Group").agg(["mean", "std", "max", "min"]).reset_index()
distance_df.columns = ["Group", "Distance_mean", "Distance_std", "Distance_max", "Distance_min"]

summary_results = pd.DataFrame({
    "Metric": ["Score", "Time", "Distance"],
    "Test Used": [score_test["test_used"], time_test["test_used"], distance_test["test_used"]],
    "p-value": [
        "<0.05" if score_test["p_value"] < 0.01 else score_test["p_value"],
        "<0.05" if time_test["p_value"] < 0.01 else time_test["p_value"],
        "<0.05" if distance_test["p_value"] < 0.01 else distance_test["p_value"]
    ],
    "Significant": [
        score_test["p_value"] < 0.05,
        time_test["p_value"] < 0.05,
        distance_test["p_value"] < 0.05
    ]
})



Mann-Whitney U Results: Statistic = 217.0000, p-value = 0.0000
Mann-Whitney U Results: Statistic = 1366.0000, p-value = 0.0000
Mann-Whitney U Results: Statistic = 1134.0000, p-value = 0.0013


In [10]:
group_stats

,Group,Score_mean,Score_std,Score_max,Score_min,Time_mean,Time_std,Time_max,Time_min,Distance_mean,Distance_std,Distance_max,Distance_min
0,HandTracking,-20.005573,33.751018,7.867548,-152.51480,14.246316,16.783415,80.321600,0.740310,1.008627,0.397666,2.110108,0.328349
1,Mouse,2.914881,1.871627,7.876531,-1.34488,2.980645,1.018363,5.300392,0.517194,0.749219,0.328794,1.739778,0.287399


In [11]:
summary_results

,Metric,Test Used,p-value,Significant
0,Score,Mann-Whitney U,<0.05,True
1,Time,Mann-Whitney U,<0.05,True
2,Distance,Mann-Whitney U,<0.05,True


In [12]:
def print_out_latex_from_pd(pd, name):
    Header = pd.columns.tolist()
    stringheader = ",".join(Header)
    print("\\begin{figure}\n",
          "\\centering\n",
          f"{pd.to_latex(index=False, float_format='%.2f', escape=False)}\n",
          "\\caption{","A table showing the",name," With the headers: ",stringheader,"}\n",
          f"\\label{{tab-{name}}}\n",
          "\\end{figure}\n")

In [13]:
print_out_latex_from_pd(summary_results, "Overall_results_Between_groups")
print_out_latex_from_pd(score_df, "Scores_Between_groups_summary")
print_out_latex_from_pd(time_df, "Time_Between_groups_summary")
print_out_latex_from_pd(distance_df, "Distance_Between_groups_summary")


\begin{figure}
 \centering
 \begin{tabular}{lllr}
\toprule
Metric & Test Used & p-value & Significant \\
\midrule
Score & Mann-Whitney U & <0.05 & True \\
Time & Mann-Whitney U & <0.05 & True \\
Distance & Mann-Whitney U & <0.05 & True \\
\bottomrule
\end{tabular}

 \caption{ A table showing the Overall_results_Between_groups  With the headers:  Metric,Test Used,p-value,Significant }
 \label{tab-Overall_results_Between_groups}
 \end{figure}

\begin{figure}
 \centering
 \begin{tabular}{lrrrr}
\toprule
Group & Score_mean & Score_std & Score_max & Score_min \\
\midrule
HandTracking & -20.01 & 33.75 & 7.87 & -152.51 \\
Mouse & 2.91 & 1.87 & 7.88 & -1.34 \\
\bottomrule
\end{tabular}

 \caption{ A table showing the Scores_Between_groups_summary  With the headers:  Group,Score_mean,Score_std,Score_max,Score_min }
 \label{tab-Scores_Between_groups_summary}
 \end{figure}

\begin{figure}
 \centering
 \begin{tabular}{lrrrr}
\toprule
Group & Time_mean & Time_std & Time_max & Time_min \\
\midrule
H